In [ ]:
import pickle
from apify_client import ApifyClient
import time
import re
from selenium.webdriver.support import expected_conditions as EC
import requests
#import json
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd

# Get the 30 first posts of the 25 players
playersList = ["vinijr", "marcelotwelve", "marega11", "pwebo99", "romelulukaku", "sterling7", "balotelliclub", "samueletoo", "toniruediger",
         "marcusrashford", "hamitaltintopofficial"]

sec_playerList = ["neymarjr", "danialves", "mahamadoudiarra_", "antony00", "luissuarez9", "diego.costa", "antogriezmann", "mouctar_dkh",
                 "mujaids4", "williaaaams11", "andressaalves9oficial", "jeffersonlerma", "samumtiti", "p.diop23"]

playersList = playersList + sec_playerList

### Trying to access instagram API and extracting the comments without success :(

In [ ]:
"""
# extract de media_id
access_token = "XXX"
shortcode = "CnxWUqyMChM"

# Construct the API endpoint URL
url = f"https://graph.instagram.com/v15.0/vinijr/media?access_token={access_token}"

# Make a GET request to the API
response = requests.get(url)
print(response)
# Parse the JSON data
#data = json.loads(response.text)

# Extract the media ID
#media_id = data['data']['id']

#print(media_id)

"""
# extract de comments using media_id 
access_token = "XXX"
media_id = "ChMVDO9s-YA"

# Construct the API endpoint URL
url = f"https://api.instagram.com/v15.0/media/{media_id}/comments?access_token={access_token}"

# Make a GET request to the API
response = requests.get(url)

print(response)

# Parse the JSON data
data = json.loads(response.text)

# Extract the comments
comments = data['data']

# Print the comments
for comment in comments:
    print(comment['text'])

### Using ApifyClient to extract info about the selected players

All commented lines have been used but are commented to avoid overwriting variables. 

In [ ]:
# Initialize the ApifyClient with API token
client = ApifyClient("XXXXX") # your apify client

# Creating a function that gets the first 30 post-ids of the passed player
def getApiInfo(player):
    lista = []
    # Prepare the actor input
    run_input = {
        "username": [player],
        "resultsLimit": 30,
    }

    # Run the actor and wait for it to finish
    run = client.actor("zuzka/instagram-post-scraper").call(run_input=run_input)

    # Fetch and print actor results from the run's dataset (if there are any)
    for item in client.dataset(run["defaultDatasetId"]).iterate_items():
        lista.append(item)
    return lista

# Returns a dict with 30 posts of each player
def getAllInfo(players_list):
    players_info = dict()
    for player in players_list:
        tmp_list = getApiInfo(player)
        players_info[player] = tmp_list
    return players_info

In [ ]:
#players_info_new = getAllInfo(sec_playerList)

In [ ]:
# Saving all the info received from apify calls (players info) into a pickle object.
"""# Saving all the players' info to a pickle object in order to access to the data later
with open('total_players_insta.pickle', 'wb') as handle:
    pickle.dump(total_players_insta, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Checking if it's saved
with open('total_players_insta.pickle', 'rb') as handle:
    total_players_insta_test = pickle.load(handle)

print(total_players_insta == total_players_insta_test)"""

In [ ]:
# Merging two previous dicts with different data of 2 different set of players into one single collection
#total_players_insta = {**players_info,**players_info_new}
#print(total_players_insta)

In [ ]:
# Reading it once it's already stored
with open('total_players_insta.pickle', 'rb') as handle:
    players_info = pickle.load(handle)

print(players_info)

### Using chromeDriver to log in to Instagram

In [ ]:
# Open chrome
driver = webdriver.Chrome("./chromedriver_win32/chromedriver.exe")

# Open instagram's page
main_url = "https://www.instagram.com/"
driver.get(main_url)
driver.maximize_window()

# Log in, in order to be able to extend the comments with the (+) button
f=open("credentials.txt","r")
lines=f.readlines()
username=lines[0]
password=lines[1]
f.close()

# Accepting and refusing starting page buttons
cookies_button = driver.find_element(By.CLASS_NAME, '_a9--._a9_1')
cookies_button.click()

# element = WebDriverWait(driver, 5).until(lambda x: x.find_element(By.NAME, "username"))

username_input = driver.find_element(By.NAME, "username")
password_input = driver.find_element(By.NAME, "password")
username_input.send_keys(username)
password_input.send_keys(password)

driver.implicitly_wait(5)

join_button = driver.find_element(By.CLASS_NAME , '_acan._acap._acas._aj1-')
join_button.click()

In [ ]:
# Clicking pop-up buttons once logged in
not_now_button = driver.find_element(By.CLASS_NAME, "_acan._acap._acas._aj1-")
not_now_button.click()

not_now_notifications_button = driver.find_element(By.CLASS_NAME, "_a9--._a9_1")
not_now_notifications_button.click()

### Using selenium and BeautifulSoup to extract instagram comments

In [ ]:
# Getting the 30 first postURLs of username
def getPostURL_by_username(username):
    postURLs_by_username = list()
    
    for post in players_info[username]:
        postURLs_by_username.append(post['url'])
        
    return postURLs_by_username

# Creating a dict collection of  player: postURLs
def getAllPostURL(players_list):
    allPostURLs = dict()
    
    for player in players_list:
        allPostURLs[player] = getPostURL_by_username(player)
        
    return allPostURLs

allPostsURL = getAllPostURL(playersList)

In [ ]:
"""
Scrap the comments of an URL
"""
def extract_comments(player, url):
    driver.get(url)
    lista = []
    
    # Click the more comments button 60 times
    more_button = driver.find_element(By.CLASS_NAME, '_ab8w._ab94._ab99._ab9h._ab9m._ab9p._abcj._abcm')
    more_button.click()
    try:
        i = 0
        while (i < 60):
            wait = WebDriverWait(driver, 40)
            more_button = driver.find_element(By.CLASS_NAME, '_ab8w._ab94._ab99._ab9h._ab9m._ab9p._abcj._abcm')
            more_button.click()
            time.sleep(2)
            i += 1
    except:
        print("can't load more button")
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Extract the comments
    comments = soup.find_all('div', {'class': '_a9zs'})

    # Store the comments in a df row
    for comment in comments:
        lista.append([player, comment.text, 'instagram'])
        
    return lista

"""
Extract the comments of all the players in the df
"""
def extract_all_comments(player, df):
    for url in allPostsURL[player]:
        try:
            rows_list = extract_comments(player, url)

            for row in rows_list:
                df.loc[len(df)] = row
        except:
            print("comments not found for this post")
    return df

# Given a list of players, iterate over the post codes and exract the comments
def extract_all(lista):
    df = pd.DataFrame(columns=['player', 'comment', 'media'])
    
    for player in lista:
        try:
            df = extract_all_comments(player, df)
        except:
            print("comments not found for player", player)
    
    return df
    

# Now it's possible to pass a complete list of players and It won't fail (no need slicing)
try1 = playersList[:3] # 3 first players
try2 = playersList[3:6]
try3 = playersList[6:9]
try4 = playersList[9:12]
try5 = playersList[12:15]
try6 = playersList[15:16]
final_try = playersList[15:] # all the players left
final_df = extract_all(final_try)
final_df.to_csv("C:/Users/jorge/TFG/Football_players_offenses/InstagramScraping/comments_dataset_10p.csv", index=False)

In [ ]:
# Check how the dataset looks after creating it with all the football players posts' comments.
final_df

In [ ]:
# Concatenate all the datasets created on different tries (extract_all)
dataset1 = pd.read_csv("InstagramScraping/comments_dataset1.csv")
dataset2 = pd.read_csv("InstagramScraping/comments_dataset2.csv")
dataset3 = pd.read_csv("InstagramScraping/comments_dataset3.csv")
dataset4 = pd.read_csv("InstagramScraping/comments_dataset4.csv")
dataset5 = pd.read_csv("InstagramScraping/comments_dataset5.csv")
dataset6 = pd.read_csv("InstagramScraping/comments_dataset_lukaku.csv")
dataset7 = pd.read_csv("InstagramScraping/comments_dataset_10p.csv")
final_dataset = pd.concat([dataset1, dataset2, dataset3, dataset4, dataset5, dataset6, dataset7])
final_dataset
dataset1

### A bit of data cleaning

In [ ]:
# Function to remove the emojis from the comments 
def remove_emojis(input_string):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs, including the white heart emoji 🤍
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001F90D"  # the white heart emoji 🤍
        u"\U0001F979"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', input_string)

# Function to remove symbols from the comments
def remove_non_letters(text):
    return "".join(c for c in text if c.isalnum() or c.isspace())

In [ ]:
# Replacing comments with emojis for '' and removing all the symbols that are not letters
final_dataset['comment'] = final_dataset['comment'].apply(remove_emojis).apply(remove_non_letters)

# Removing all the rows that are empty ('')
final_dataset.drop(final_dataset[final_dataset['comment'] == ''].index, inplace = True)

dataset = dataset.reset_index(drop=True)

final_dataset

### Saving the dataset

In [ ]:
# Commenting it to avoid overwritting the dataset
"""# Saving the complete dataset to a pickle object in order to access to the data later
with open('final_dataset.pickle', 'wb') as handle:
    pickle.dump(dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Checking if it's saved
with open('final_dataset.pickle', 'rb') as handle:
    final_dataset_test = pickle.load(handle)

print(all(dataset == final_dataset_test))

if all(dataset == final_dataset_test):
    dataset.to_csv("C:/Users/jorge/TFG/Football_players_offenses/CleanTranslateDataset/final_dataset.csv", index=False)"""

In [ ]:
import pickle

# Reading the dataset once it's already stored
with open('CleanTranslateDataset/final_dataset.pickle', 'rb') as handle:
    dataset = pickle.load(handle)

dataset.head(10)